## Text similarity

### 数据预处理

In [1]:
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#load stop words
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [i[:-1] for i in f.readlines()]
# print(stopwords)

In [2]:
#load data
news = pd.read_csv('sqlResult.csv', encoding='gb18030')
print(news.shape)
print(news.head())

#clean data
drop_content = news.dropna(subset=['content'])
drop_all = news.dropna()
print(drop_all.shape)
print(drop_content.shape)
news = drop_content

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [3]:
for n in news['source']:
    print(n)

快科技@http://www.kkj.cn/
快科技@http://www.kkj.cn/
快科技@http://www.kkj.cn/
新华社
深圳大件事
中国证券报?中证网
威锋网@http://www.feng.com/
中国证券报?中证网
荆楚网-楚天都市报
中国证券报?中证网
证券时报网
证券时报网
中国新闻网
中国证券报?中证网
国际在线
中国证券网
solidot@
新华网
中国证券报?中证网
中国证券报?中证网
泡泡网
中国证券报?中证网
威锋网@http://www.feng.com/
中国证券报?中证网
凤凰体育
新华网
雷锋网@
新华网
中超球评
中国经济周刊
人民日报
央广网
快科技@http://www.kkj.cn/
体育君
快科技@http://www.kkj.cn/
新浪体育
雷锋网@http://www.leiphone.com/
新浪体育
体育君
新华社
雷锋网@http://www.leiphone.com/
广州日报
TechWeb@
足球小志
TechWeb@
人民网
中国经济周刊
时代财经@
新疆日报
中超球评
中关村在线@
新华网
网易科技@http://tech.163.com/
央广网
封面新闻
央广网
央视新闻
央广网
澎湃新闻网@
澎湃新闻网
中青在线
央视新闻客户端@
中国新闻网
人民日报
千龙网
新华社@
央广网
中国新闻网
中国新闻网
新浪体育
新浪综合@
中新网
央广网
GamerSky游民星空@http://www.gamersky.com/
中国经济网
cnBeta.COM
cnBeta.COM
热点体育
央广网
cnBeta.COM
新疆日报
新浪体育
solidot@
枪炮篮球PiU
中国新闻网
央广网
中国网财经
知客数码@http://www.zaeke.com/article-7066-1.html
中国网财经
中国网财经
新疆日报
于斌@http://www.chaoqi.net/
中国新闻网
霍寿喜
中国新闻网
新华社
中国新闻网
网易科学人@
中国新闻网
中国新闻网
中国经济网
新疆日报
新华网
凤凰体育
看看新闻KNEWS
新浪综合
中国网财经
新浪体育
最后一公里
中国新闻网
中国证券报?中证网
新华社
中国经济网
中国网财经
凤凰体育
中国网财经
新疆日报
中国新闻网
中国网财经

参考消息网
环球网
央视网
新华社新媒体专线
央视网
新浪体育
上海江森自控国际蓄电池有限公司
海外探客
未来网
凤凰财经WEMONEY
中国新闻网
新浪体育
未来网
旅业传媒
人民网
智通财经
人民网
每日经济新闻公号
中国台湾网
每日经济新闻
中国台湾网
国际在线
中国台湾网
文思海辉
轻松一刻工作室
参考消息网
中国台湾网
智东西
中国台湾网
现代快报
中国台湾网
央视
中国网
中国网
央视财经
Mesosphere
中国网
西楚网
中国台湾网
环球时报-环球网
凤凰财经WEMONEY
正义网
梨视频
正义网
正义网
中国证券报?中证网
正义网
大白新闻
上海证券报
正义网
凤凰财经WEMONEY
中国证券报?中证网
正义网
环球时报-环球网
中国新闻网
证券时报
湘东检察
中国新闻网
21世纪经济报道
FUN来了
人民网
21世纪经济报道
环球网
21世纪经济报道
中国新闻网
每日人物
上海证券报
参考消息网
每日经济新闻
凤凰财经WEMONEY
人民网
上海证券报
央视
中国新闻网
中国证券报?中证网
中国新闻网
人民网
证券时报
央广网
飞象网
证券时报
商业人物
央广网
凤凰财经WEMONEY
信息时报
中国新闻网
读懂新三板
新浪体育
中国新闻网
证券时报
环球时报-环球网
参考消息网
证券日报
参考消息网
证券时报网
央视新闻
中国网
新浪财经
中国新闻网
证券时报
新浪科技@http://tech.sina.com.cn/
参考消息网
凤凰财经WEMONEY
搜狐
千龙网
中国新闻网
中国网财经
参考消息网
cnBeta.COM
证券时报
人民网
证券时报
FM93交通之声
人民网
中国证券报?中证网
中国新闻网
中国证券报?中证网
中国新闻网
中国新闻网
中国经济网
中国证券报?中证网
cnBeta.COM
中国证券报?中证网
参考消息网
凤凰财经WEMONEY
读懂新三板
新华社
参考消息网
证券时报
参考消息网
中国网财经
中国新闻网
参考消息网
证券时报
参考消息网
cnBeta.COM
搜狐
参考消息网
证券日报
搜狐
央广网
凤凰财经WEMONEY
半岛晨报
搜狐
中国新闻网
搜狐
科技成果转化联盟
中国新闻网
国际金融报
雷锋网
环球时报-环球网
千龙网
每日经济新闻
中国广播网
中国新闻网
中国证券报
快科技@http://www

法制网
华商报
新华网
法制网
新浪体育
中新网@
中国证券报?中证网
法制网
新浪体育
环球网
中国证券报?中证网
法制网
中国证券报?中证网
新浪美股@
参考消息网
中国证券报?中证网
新华网
中国青年报
中国证券报?中证网
参考消息网
环球时报
中国证券报?中证网
新华网
环球时报-环球网
中国证券报?中证网
环球网
光明网
环球时报-环球网
中国证券报?中证网
海外网
TechWeb@
光明网
一财网
光明网
梅州广播电视台
光明网
新浪科技
光明网
中国证券报?中证网
新华社
中国证券报?中证网
光明网
环球网
中国证券报?中证网
人民网
21世纪经济报道
地震局网站
中国新闻网
中国新闻网
cnBeta.COM
中国证券报?中证网
环球网
中国新闻网
中国证券报?中证网
新浪综合
千龙网
中国证券报?中证网
中国新闻网
中国证券报?中证网
中国新闻网
中国证券报?中证网
环球时报
中国新闻网
新华社新媒体专线
中国证券报?中证网
新华社
中国新闻网
新浪体育
中国证券报?中证网
环球时报
中国新闻网
环球时报
中国新闻网
中国证券报?中证网
第一财经@http://tech.163.com/17/0620/14/CNCOOB6800097U7R.html
参考消息网
中国证券报?中证网
环球时报-环球网
参考消息网
中国证券报?中证网
环球网
参考消息网
中国证券报?中证网
环球网
中国新闻网
中国证券报?中证网
新浪综合
央广网
中国证券报?中证网
央视新闻
网易科技@http://tech.163.com/
证券日报
参考消息网
第一财经
南方都市报
中国新闻网
凤凰财经综合
北京商报
中国新闻网
新京报
环球网
中国新闻网
中国证券报?中证网
王三三
中国新闻网
上海证券报
环球网
人民网
中国证券网
新浪娱乐
千龙网
华商网
nan
凤凰网科技@
中国新闻网
中国新闻网
上海证券报
证券时报网
中国新闻网
环球时报
凤凰国际iMarkets
中国证券网
中国新闻网
上海证券报
环球网
中国新闻网
21世纪经济报道
中国新闻网
中国新闻网
中国证券报
中国新闻网
新浪娱乐
中国证券报?中证网
中国证券报?中证网
每日经济新闻
cnBeta.COM
每日经济新闻
凤凰财经
上海证券报
澎湃新闻网
环球网
广州日报第ZSA15版
证券时报
中国新闻网


广州日报第01版
外交部网站
长江日报第1版
郑州日报第01版
海南日报第001版
成都商报第01版
南方日报第01版
广州日报第01版
长江日报第1版
搜狐娱乐
郑州日报第01版
海南日报第001版
成都商报第01版
南方日报第01版
广州日报第01版
长江日报第1版
郑州日报第01版
广州日报第01版
成都商报第01版
中国新闻网
海南日报第001版
南方日报第01版
乐视体育
郑州日报第01版
长江日报第1版
海南日报第001版
郑州日报第01版
新华社
新华社
新华社
网络
天山网原创
安徽网
中国新闻网
新华社
澎湃新闻
央广网
中国体育报
虎扑足球
虎扑篮球
中国新闻社
央广网
中国新闻网
中国新闻网
环球网
中国新闻网
华夏经纬网
华夏经纬网
华夏经纬网
华夏经纬网
海外网
中国台湾网
中国新闻网
中国台湾网
中国新闻网
中国台湾网
中国台湾网
中国台湾网
中国台湾网
中国新闻网
中国台湾网
环球时报
解放军报
环球时报
环球时报
环球网
解放军报
环球时报
环球时报
海外网
东线瞭望
环球网
人民网
环球时报
环球网
环球时报
环球时报
环球时报
环球时报
环球时报
环球时报
环球网
人民网
环球网
环球时报
人民日报
环球网
侠客岛
央视新闻客户端
科技日报
新华网
新华社
新华社
新华社
中国新闻网
中国新闻网
中国新闻网
中国新闻网
新华社
中国网
中国新闻网
新华社
中国新闻网
新华社
新华社
新华社
新华社
新华社
中国新闻网
新华社
中国新闻网
外事儿
中国新闻网
中国新闻网
新华网
新华社
中国新闻网
新华社
中央纪委监察部网站
南方网
天山网
央广网
天山网
央广网
央广网
央广网
央广网
央广网
多彩贵州网
央广网
央广网
央广网
中国政府网
吉网
中新网
新华社
新华社
央广网
央广网
多彩贵州网
央广网
央广网
天山网原创
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
环球网
老邓侃球
中新网
体育疯
中国新闻网
中国新闻网
新华社
肆客足球App
5U体育
凯发精英体育
新华社
5U体育
环球网
环球网
环球网
化妆真的简单
环球网
海报网
环球网
ELLEMEN睿士
维欧服装设计留学作品集
Bomoda摩登系
悦

新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


...

新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社
新华社


In [4]:
def split_text(text):
    text = text.replace(' ', '').replace('\n', '')
    text = jieba.cut(text.strip())
    result = ' '.join([w for w in text if w not in stopwords])
    return result
# print(split_text(news.iloc[0].content))

In [5]:
import pickle, os
if not os.path.exists('corpus.pkl'):
    corpus = list(map(split_text, [str(i) for i in news.content]))
    print(corpus[0])
    with open('corpus.pkl', 'wb') as f:
        pickle.dump(corpus, f)
else:
    with open('corpus.pkl', 'rb') as f:
        corpus = pickle.load(f)
    
#calc tf-idf matrix
count_vectorizer = CountVectorizer(encoding='gb18030', min_df=0.015)
tfidf_transformer = TfidfTransformer()
count_vector = count_vectorizer.fit_transform(corpus)
tfidf = tfidf_transformer.fit_transform(count_vector)
print(tfidf.shape)


(87054, 884)


### 分类方法寻找新华社文章

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


# 标记新华社文章的index
label = list(map(lambda source: 1 if '新华社' in str(source) else 0, news.source))

# split dataset
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size=0.2)
# 贝叶斯分类（新华社 vs All）
clf = MultinomialNB()
clf.fit(X_train, y_train)
#predict
y_predict = clf.predict(tfidf.toarray())
labels = np.array(label)

compare_news_index = pd.DataFrame({'prediction': y_predict, 'labels': labels})
# false positive
fake_news_index = compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index
# 新华社index
xinhua_news_index = compare_news_index[(compare_news_index['labels'] == 1)].index
print('suspect:', len(fake_news_index))


suspect: 2802


In [7]:
print("false positive:", len(fake_news_index))
print("true positive:", len(xinhua_news_index))
print("positive:", len(fake_news_index) + len(xinhua_news_index))
print("样本总量:", len(news))

false positive: 2802
true positive: 78674
positive: 81476
样本总量: 87054


### 聚类方法寻找同类文章并建立映射

In [8]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer

#normalize
norm = Normalizer()
scaled_array = norm.fit_transform(tfidf.toarray())
#clustering
if not os.path.exists('label.pkl'):
    # 取25个中心
    kmeans = KMeans(n_clusters=25)
    k_labels = kmeans.fit_predict(scaled_array)
    with open('label.pkl', 'wb') as f:
        pickle.dump(k_labels, f)
    print('k_labels.shape', k_labels.shape)
else:
    with open('label.pkl', 'rb') as f:
        k_labels = pickle.load(f)
# k_labels[i]表示第i个news所属的类的idx

In [9]:
# mapping from idx to class
id_class = {index:class_ for index, class_ in enumerate(k_labels)}
# mapping from class to a set of indices
if not os.path.exists('id_class.pkl'):
    from collections import defaultdict
    class_id = defaultdict(set)
    for index, class_ in id_class.items():
        if index in xinhua_news_index.tolist():
            class_id[class_].add(index)        
    with open('class_id.pkl', 'wb') as f:
        pickle.dump(class_id, f)
else:
    with open('class_id.pkl', 'rb') as f:
        class_id = pickle.load(f)

### 在同类文章中查重

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
# find similar
def find_similar_text(cpindex, top=10):
    # 计算目标与同类文章的cos similarity
    dist_dict = {i:cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 返回topN
    return sorted(dist_dict.items(), key=lambda x:x[1][0], reverse=True)[:top]

In [11]:
# 待查重文章idx
cpindex = 15
similar_list = find_similar_text(cpindex)
print('checking:\n', news.iloc[cpindex].content)
print(similar_list)
similar2 = similar_list[0][0]
print('original\n', news.iloc[similar2].content)
for i in similar_list:
    print('suspect source:\n', news.iloc[i[0]].content)

checking:
 　　中国证券网讯（记者 费天元）沪深两市周五临近午盘出现快速跳水，沪指连续跌穿5日、10日及20日均线。截至午间收盘，上证指数报3124.23点，下跌0.74%；深证成指报10219.41点，下跌0.45%；创业板指报1780.20点，下跌1.01%。
　　盘面上，部分地产板块权重股早盘异动，万科A上涨6.58%，保利地产上涨2.50%。石墨烯板块表现强势，碳元科技涨停，中科电气上涨8.39%。钛白粉板块同样涨幅靠前。
　　跌幅方面，可燃冰概念领跌，恒泰艾普下跌8.90%，石化机械、潜能恒信跌逾6%。网络游戏板块深度回调，天沃科技下跌8.34%，天润数娱下跌6.49%。雄安概念重挫，中持股份、京汉股份等跌逾7%。粤港澳、上海国资板块同样跌幅靠前。

[(41117, array([[0.58861671]])), (32255, array([[0.57943666]])), (60673, array([[0.57366601]])), (41123, array([[0.57257852]])), (68759, array([[0.57166043]])), (78875, array([[0.57016106]])), (7997, array([[0.56228704]])), (23341, array([[0.55702567]])), (61814, array([[0.55050556]])), (82210, array([[0.54608038]]))]
original
 　　新华社芝加哥４月２７日电（记者汪平）纽约商品交易所黄金期货市场交投最活跃的６月黄金期价２７日比前一交易日上涨１．７美元，收于每盎司１２６５．９美元，涨幅为０．１３％。\n　　市场分析人士说，在经过连续３个交易日的下跌后，黄金期价技术性反弹，最终以小幅上涨收盘。\n　　２７日其他贵金属方面，７月交割的白银期货价格下跌９．７美分，收于每盎司１７．３３４美元，跌幅为０．５６％；７月交割的白金期货价格上涨０．７美元，收于每盎司９４８．８美元，涨幅为０.０７％。（完）\n
suspect source:
 　　新华社芝加哥４月２７日电（记者汪平）纽约商品交易所黄金期货市场交投最活跃的６月黄金期价２７日比前一交易日上涨１．７美元，收于每盎司１２６５．９美元，涨